In [1]:
import geopandas as gpd
import pandas as pd
import verde as vd
import numpy as np
from shapely import geometry
import my_funcs as mf

import matplotlib.pyplot as plt
%matplotlib widget

gdb='/home/ggrl/geodatabase/'

In [12]:
g_rparaim = pd.read_csv(gdb+'geof/recortado/g_rparaim')

In [2]:
g_varginha = pd.read_csv(gdb+'geof/recortado/g_varginha')

In [3]:
l_100k =  mf.litologia('l_100k')

In [4]:
rparaim = mf.mapa(l_100k,'Rio Paraim')

mansidao = mf.mapa(l_100k, 'Mansidão')

macaubas = mf.mapa(l_100k, 'Macaúbas')

varginha = mf.mapa(l_100k,'Varginha - UFRJ')

In [169]:
base = rparaim.plot('SIGLA')
mansidao.plot('SIGLA',ax=base)
#macaubas.plot('SIGLA',ax=base)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [8]:
rparaim.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [161]:
rparaim = rparaim.to_crs('EPSG:32723')
varginha = varginha.to_crs('EPSG:32723')
mansidao = mansidao.to_crs('EPSG:32723')

In [162]:
rparaim.plot('SIGLA')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [13]:
g_rparaim

,X,Y,MDT,KPERC,eU,eth,CTCOR,LONGITUDE,LATITUDE
0,500020.25,8786155.43,477.68,0.35,1.25,16.12,1182.85,-44.999815,-10.980692
1,500103.83,8786158.77,476.83,0.63,1.24,14.00,1143.89,-44.999050,-10.980662
2,500187.24,8786161.64,476.58,1.02,1.38,12.02,1165.74,-44.998286,-10.980636
3,500270.50,8786163.59,470.91,1.41,1.51,11.31,1257.68,-44.997524,-10.980618
4,500353.54,8786165.69,464.63,1.71,1.65,11.14,1357.00,-44.996764,-10.980599
...,...,...,...,...,...,...,...,...,...
81720,554618.93,8838871.32,477.57,0.96,1.98,17.01,1507.21,-44.500850,-10.503529
81721,554619.00,8838949.65,471.69,0.95,1.47,15.02,1327.04,-44.500850,-10.502821
81722,554618.94,8839028.43,463.98,0.80,1.17,12.29,1087.91,-44.500852,-10.502108
81723,554618.52,8839107.80,454.86,0.63,1.10,9.72,881.47,-44.500857,-10.501391


In [6]:
g_varginha

,CTCOR,eTh,eU,KPERC,LATITUDE,LONGITUDE,MDT,X,Y
0,1143.73,14.0,0.6,0.767,-21.500178,-45.498803,841.27,448358.28,7622423.14
1,1081.94,13.2,0.7,0.672,-21.500752,-45.498801,840.90,448358.69,7622359.61
2,1037.61,12.9,0.7,0.559,-21.501328,-45.498800,845.45,448359.00,7622295.86
3,1012.03,12.9,0.7,0.503,-21.501909,-45.498800,850.90,448359.20,7622231.55
4,1037.67,13.4,0.6,0.539,-21.502495,-45.498801,849.13,448359.31,7622166.69
...,...,...,...,...,...,...,...,...,...
78239,1678.03,23.4,0.8,0.144,-21.523203,-45.496703,876.74,448583.90,7619875.37
78240,1793.02,25.0,0.8,0.210,-21.523223,-45.497524,884.01,448498.88,7619872.89
78241,1860.36,25.8,0.7,0.292,-21.523240,-45.498342,880.68,448414.17,7619870.74
78242,1886.30,25.9,0.6,0.387,-21.523255,-45.499159,871.14,448329.57,7619868.81


In [14]:
chain_list = ['CTCOR', 'KPERC', 'MDT', 'eU', 'eth']

grids = {'CTCOR':(),'KPERC':(), 'MDT':(), 'eU':(), 'eth':()}

coordinates = (g_rparaim.X.values, g_rparaim.Y.values)

for i in chain_list:
    print(i)
    
    chain = vd.Chain([
        ('trend', vd.Trend(degree=1)),
        ('reduce', vd.BlockReduce(np.median, spacing=1000)),
        ('spline', vd.Spline())
    ])
    
    chain.fit(coordinates, g_rparaim[i])
    
    grid = chain.grid(spacing=200, data_names=[i],pixel_register=True)
    grids[i] = vd.distance_mask(coordinates, maxdist=1000, grid= grid)

CTCOR
KPERC
MDT
eU
eth


In [15]:
data = list()
for i in chain_list:
    df = grids[i].to_dataframe()
    data.append(df[i])

In [16]:
geof = pd.concat(data,axis=1, join='inner')

In [17]:
geof

CTCOR     KPERC         MDT        eU  \
easting       northing                                                    
500105.462198 8.784085e+06  1659.760751  1.120279  514.907207  2.018264   
              8.784285e+06  1632.120183  1.106715  513.074245  2.022793   
              8.784484e+06  1591.655859  1.087899  509.603425  2.028829   
              8.784684e+06  1537.350905  1.064791  504.364548  2.034102   
              8.784884e+06  1472.429274  1.039333  497.970216  2.035450   
...                                 ...       ...         ...       ...   
554593.697802 8.838405e+06  1164.247622  0.729238  487.197021  2.040467   
              8.838605e+06  1167.331706  0.755726  481.847074  2.063674   
              8.838805e+06  1169.905155  0.778491  477.275848  2.058947   
              8.839004e+06  1170.249431  0.793855  473.794391  2.015057   
              8.839204e+06  1168.968982  0.802948  471.058906  1.950131   

                                  eth  
easting       northing                 
500105.462198 8.784085e+06  16.542191  
              8.784285e+06  16.315067  
              8.784484e+06  15.941735  
              8.784684e+06  15.410646  
              8.784884e+06  14.764774  
...                               ...  
554593.697802 8.838405e+06  11.906200  
              8.838605e+06  11.927215  
              8.838805e+06  11.981648  
              8.839004e+06  12.065598  
              8.839204e+06  12.162775  

[75621 rows x 5 columns]

In [18]:
geof.reset_index(inplace=True)
geof

,easting,northing,CTCOR,KPERC,MDT,eU,eth
0,500105.462198,8.784085e+06,1659.760751,1.120279,514.907207,2.018264,16.542191
1,500105.462198,8.784285e+06,1632.120183,1.106715,513.074245,2.022793,16.315067
2,500105.462198,8.784484e+06,1591.655859,1.087899,509.603425,2.028829,15.941735
3,500105.462198,8.784684e+06,1537.350905,1.064791,504.364548,2.034102,15.410646
4,500105.462198,8.784884e+06,1472.429274,1.039333,497.970216,2.035450,14.764774
...,...,...,...,...,...,...,...
75616,554593.697802,8.838405e+06,1164.247622,0.729238,487.197021,2.040467,11.906200
75617,554593.697802,8.838605e+06,1167.331706,0.755726,481.847074,2.063674,11.927215
75618,554593.697802,8.838805e+06,1169.905155,0.778491,477.275848,2.058947,11.981648
75619,554593.697802,8.839004e+06,1170.249431,0.793855,473.794391,2.015057,12.065598


In [19]:
geof['geometry'] = [geometry.Point(x,y) for x, y in zip(geof['easting'], geof['northing'])]

In [20]:
gdf = gpd.GeoDataFrame(geof,crs=32723)
gdf = gdf.set_crs(32723, allow_override=True)

In [21]:
gdf.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [22]:
gdf

,easting,northing,CTCOR,KPERC,MDT,eU,eth,geometry
0,500105.462198,8.784085e+06,1659.760751,1.120279,514.907207,2.018264,16.542191,POINT (500105.462 8784085.083)
1,500105.462198,8.784285e+06,1632.120183,1.106715,513.074245,2.022793,16.315067,POINT (500105.462 8784284.789)
2,500105.462198,8.784484e+06,1591.655859,1.087899,509.603425,2.028829,15.941735,POINT (500105.462 8784484.495)
3,500105.462198,8.784684e+06,1537.350905,1.064791,504.364548,2.034102,15.410646,POINT (500105.462 8784684.201)
4,500105.462198,8.784884e+06,1472.429274,1.039333,497.970216,2.035450,14.764774,POINT (500105.462 8784883.907)
...,...,...,...,...,...,...,...,...
75616,554593.697802,8.838405e+06,1164.247622,0.729238,487.197021,2.040467,11.906200,POINT (554593.698 8838405.093)
75617,554593.697802,8.838605e+06,1167.331706,0.755726,481.847074,2.063674,11.927215,POINT (554593.698 8838604.799)
75618,554593.697802,8.838805e+06,1169.905155,0.778491,477.275848,2.058947,11.981648,POINT (554593.698 8838804.505)
75619,554593.697802,8.839004e+06,1170.249431,0.793855,473.794391,2.015057,12.065598,POINT (554593.698 8839004.211)


In [148]:
base = rparaim.plot('SIGLA')
gdf.plot(ax=base,markersize=0.5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [45]:
gdf.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [117]:
rparaim['SIGLA'].iloc[rparaim.distance(gdf.geometry[0]).idxmin()]

IndexError: single positional indexer is out-of-bounds

In [143]:
# calculo da menor distancia entre o primeiro ponto do GeoDataFrame e uma geometria do mapa
[rparaim.distance(gdf.geometry[0]).idxmin()]  # .idxmin() Retorna o indice do menor valor

[36]

In [127]:
rparaim = rparaim.reset_index()

In [137]:
rparaim = rparaim.drop('index',axis=1)

In [141]:
geof['closest_unid'] = gdf['geometry'].apply(lambda x: rparaim['SIGLA'].iloc[rparaim.distance(x).idxmin()]) # .idxmin() Retorna o indice do menor valor 

In [142]:
geof.to_csv(gdb+'')

,easting,northing,CTCOR,KPERC,MDT,eU,eth,geometry,closest_unid
0,500105.462198,8.784085e+06,1659.760751,1.120279,514.907207,2.018264,16.542191,POINT (500105.462 8784085.083),NP1rp3
1,500105.462198,8.784285e+06,1632.120183,1.106715,513.074245,2.022793,16.315067,POINT (500105.462 8784284.789),NP1rp3
2,500105.462198,8.784484e+06,1591.655859,1.087899,509.603425,2.028829,15.941735,POINT (500105.462 8784484.495),NP1rp3
3,500105.462198,8.784684e+06,1537.350905,1.064791,504.364548,2.034102,15.410646,POINT (500105.462 8784684.201),NP1rp3
4,500105.462198,8.784884e+06,1472.429274,1.039333,497.970216,2.035450,14.764774,POINT (500105.462 8784883.907),NP1rp3
...,...,...,...,...,...,...,...,...,...
75616,554593.697802,8.838405e+06,1164.247622,0.729238,487.197021,2.040467,11.906200,POINT (554593.698 8838405.093),NP1rp3
75617,554593.697802,8.838605e+06,1167.331706,0.755726,481.847074,2.063674,11.927215,POINT (554593.698 8838604.799),NP1rp3
75618,554593.697802,8.838805e+06,1169.905155,0.778491,477.275848,2.058947,11.981648,POINT (554593.698 8838804.505),NP1rp3
75619,554593.697802,8.839004e+06,1170.249431,0.793855,473.794391,2.015057,12.065598,POINT (554593.698 8839004.211),NP1rp3


In [151]:
rparaim = rparaim.to_crs(4326)

In [ ]:
l_1kk = mf.litologia('l_1kk')

In [153]:
base = l_1kk.plot('SIGLA')
rparaim.boundary.plot(ax=base)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [173]:
plt.figure('Terrain')
grids['MDT'].MDT.plot(cmap='terrain')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(500005.29999999993, 554693.8599999999, 8783985.23, 8839303.77)

In [174]:
plt.figure('Contagem Total')
grids['CTCOR'].CTCOR.plot(cmap='hsv')
plt.axis('scaled')

<ipython-input-174-d2c99bc4b4fe>:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure('Contagem Total')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(500005.29999999993, 554693.8599999999, 8783985.23, 8839303.77)